In [2]:
import os
from PIL import Image
from PIL.ExifTags import TAGS

# Your image path
image_file = "/home/shashank/Desktop/Mini_project /IMG_9394.jpg"

# Open the image file
image = Image.open(image_file)

# Make a collection of properties and values corresponding to your image.
exif = {}
if image._getexif() is not None:
    for tag, value in image._getexif().items():
        if tag in TAGS:
            exif[TAGS[tag]] = value

if "GPSInfo" in exif:
    gps_info = exif["GPSInfo"]

    def convert_to_degrees(value):
        """
        Helper function to convert the GPS coordinates stored in the EXIF to degrees in float format.

        Args:
            value (tuple): The GPS coordinate as a tuple (degrees, minutes, seconds)

        Returns:
            float: The coordinate in degrees
        """
        d = float(value[0])
        m = float(value[1])
        s = float(value[2])
        return d + (m / 60.0) + (s / 3600.0)

    # Convert latitude and longitude to degrees
    lat = convert_to_degrees(gps_info[2])
    lon = convert_to_degrees(gps_info[4])
    lat_ref = gps_info[1]
    lon_ref = gps_info[3]

    # Adjust the sign of the coordinates based on the reference (N/S, E/W)
    if lat_ref != "N":
        lat = -lat
    if lon_ref != "E":
        lon = -lon

    # Format the GPS coordinates into a human-readable string
    geo_coordinate = "{0}° {1}, {2}° {3}".format(lat, lat_ref, lon, lon_ref)
    print(geo_coordinate)

    # Create a Google Maps link
    google_maps_link = f"https://www.google.com/maps?q={lat},{lon}"
    print(f"Google Maps link: {google_maps_link}")
else:
    print("No GPS information found.")

12.940794444444444° N, 77.56542222222222° E
Google Maps link: https://www.google.com/maps?q=12.940794444444444,77.56542222222222


In [4]:
    import os
    import csv
    from PIL import Image
    from PIL.ExifTags import TAGS
    from ultralytics import YOLO
    from datetime import datetime
    import folium
    import cv2

    # Paths
    csv_file = "pothole_detections.csv"
    html_map_file = "pothole_map.html"
    output_folder = "annotated_images"
    os.makedirs(output_folder, exist_ok=True)

    # Initialize YOLOv8 Model
    model = YOLO("yolov8n.pt")  # Replace with your trained YOLO model

    def extract_gps(image_file):
        """
        Extract GPS coordinates from the EXIF metadata of an image.
        """
        try:
            image = Image.open(image_file)
            exif = {}
            if image._getexif() is not None:
                for tag, value in image._getexif().items():
                    if tag in TAGS:
                        exif[TAGS[tag]] = value

            if "GPSInfo" in exif:
                gps_info = exif["GPSInfo"]

                def convert_to_degrees(value):
                    d = float(value[0])
                    m = float(value[1])
                    s = float(value[2])
                    return d + (m / 60.0) + (s / 3600.0)

                lat = convert_to_degrees(gps_info[2])
                lon = convert_to_degrees(gps_info[4])
                lat_ref = gps_info[1]
                lon_ref = gps_info[3]

                if lat_ref != "N":
                    lat = -lat
                if lon_ref != "E":
                    lon = -lon

                return lat, lon
            else:
                print("No GPS information found.")
                return None, None
        except Exception as e:
            print(f"Error extracting GPS: {e}")
            return None, None

    def detect_potholes(image_file):
        """
        Detect potholes in an image using YOLOv8.
        """
        results = model(image_file)
        detections = []

        # Annotate image with detections
        frame = cv2.imread(image_file)
        for result in results:
            for box in result.boxes:
                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()  # Bounding box
                confidence = box.conf.cpu().numpy()        # Confidence score
                detections.append((x1, y1, x2, y2, confidence.item()))  # Store confidence as scalar
                # Draw bounding box on the image
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
                cv2.putText(frame, f"Conf: {confidence.item():.2f}", (int(x1), int(y1) - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

        # Save annotated image
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        annotated_image_path = os.path.join(output_folder, f"annotated_{timestamp}.jpg")
        cv2.imwrite(annotated_image_path, frame)

        return detections, annotated_image_path

    def log_to_csv(latitude, longitude, confidence, image_path):
        """
        Log detection details to a CSV file.
        """
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        with open(csv_file, "a", newline="") as file:
            writer = csv.writer(file)
            writer.writerow([latitude, longitude, confidence, timestamp, image_path])

    def create_map(csv_file, output_html):
        """
        Create an interactive HTML map from logged CSV data.
        """
        data = []
        with open(csv_file, "r") as file:
            reader = csv.reader(file)
            for row in reader:
                data.append(row)

        if data:
            map_center = [float(data[0][0]), float(data[0][1])]
            m = folium.Map(location=map_center, zoom_start=15)

            for row in data:
                latitude, longitude, confidence, timestamp, image_path = row
                popup_text = f"Confidence: {confidence}<br>Timestamp: {timestamp}<br>Image: {image_path}"
                folium.Marker([float(latitude), float(longitude)], popup=popup_text).add_to(m)

            m.save(output_html)
            print(f"Map saved as {output_html}")
        else:
            print("No data to visualize.")

    # Main Workflow
    def main(image_file):
        # Step 1: Extract GPS coordinates
        lat, lon = extract_gps(image_file)
        if lat is None or lon is None:
            print("Skipping image due to missing GPS data.")
            return

        print(f"GPS Coordinates: Latitude={lat}, Longitude={lon}")

        # Step 2: Detect potholes
        detections, annotated_image_path = detect_potholes(image_file)
        if not detections:
            print("No potholes detected.")
            return

        # Step 3: Log to CSV
        for _, _, _, _, confidence in detections:
            log_to_csv(lat, lon, confidence, annotated_image_path)

        # Step 4: Create Map
        create_map(csv_file, html_map_file)

    if __name__ == "__main__":
        # Example image path
        image_file = "/home/shashank/Desktop/Mini_project /IMG_9394.jpg"
        main(image_file)


GPS Coordinates: Latitude=12.940794444444444, Longitude=77.56542222222222

image 1/1 /home/shashank/Desktop/Mini_project /IMG_9394.jpg: 480x640 1 mouse, 1 keyboard, 47.1ms
Speed: 1.9ms preprocess, 47.1ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)


ValueError: too many values to unpack (expected 5)

In [2]:
import os
import csv
from PIL import Image
from PIL.ExifTags import TAGS
from ultralytics import YOLO
from datetime import datetime
import folium
import cv2

# Paths
csv_file = "pothole_detections2.csv"
html_map_file = "pothole_map2.html"
output_folder = "annotated_images"
os.makedirs(output_folder, exist_ok=True)

# Initialize YOLOv8 Model
model = YOLO("/home/shashank/Desktop/Mini_project /runs/detect/pothole_detection9/weights/best.pt")  # Replace with your trained YOLO model

def extract_gps(image_file):
    """
    Extract GPS coordinates from the EXIF metadata of an image.
    """
    try:
        image = Image.open(image_file)
        exif = image._getexif()
        if exif is None:
            print("No EXIF metadata found.")
            return None, None
        
        exif_data = {TAGS.get(tag, tag): value for tag, value in exif.items()}
        gps_info = exif_data.get("GPSInfo")

        if gps_info is None:
            print("No GPS information found.")
            return None, None

        def convert_to_degrees(value):
            return float(value[0]) + (float(value[1]) / 60.0) + (float(value[2]) / 3600.0)

        lat = convert_to_degrees(gps_info[2])
        lon = convert_to_degrees(gps_info[4])
        if gps_info[1] != "N":
            lat = -lat
        if gps_info[3] != "E":
            lon = -lon

        return lat, lon
    except Exception as e:
        print(f"Error extracting GPS: {e}")
        return None, None

def detect_potholes(image_file):
    """
    Detect potholes in an image using YOLOv8.
    """
    results = model(image_file)
    detections = []

    # Annotate image with detections
    frame = cv2.imread(image_file)
    if frame is None:
        print(f"Error loading image: {image_file}")
        return [], None

    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()  # Bounding box
            confidence = box.conf.cpu().numpy()        # Confidence score
            detections.append((x1, y1, x2, y2, confidence.item()))  # Store confidence as scalar
            
            # Draw bounding box on the image
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
            cv2.putText(frame, f"Conf: {confidence.item():.2f}", (int(x1), int(y1) - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    # Save annotated image
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    annotated_image_path = os.path.join(output_folder, f"annotated_{timestamp}.jpg")
    cv2.imwrite(annotated_image_path, frame)

    return detections, annotated_image_path

def log_to_csv(latitude, longitude, confidence, image_path):
    """
    Log detection details to a CSV file.
    """
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    row_data = [latitude, longitude, confidence, timestamp, image_path]

    # Debug: Print what is being logged
    print(f"Logging to CSV: {row_data}")

    with open(csv_file, "a", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(row_data)

def create_map(csv_file, output_html):
    """
    Create an interactive HTML map from logged CSV data.
    """
    data = []
    with open(csv_file, "r") as file:
        reader = csv.reader(file)
        for row in reader:
            print(f"Row: {row}")  # Debugging step
            if len(row) == 5:  # Ensure only correctly formatted rows are processed
                data.append(row)
            else:
                print(f"Skipping malformed row: {row}")

    if data:
        map_center = [float(data[0][0]), float(data[0][1])]
        m = folium.Map(location=map_center, zoom_start=15)

        for row in data:
            latitude, longitude, confidence, timestamp, image_path = row
            popup_text = f"Confidence: {confidence}<br>Timestamp: {timestamp}<br>Image: {image_path}"
            folium.Marker([float(latitude), float(longitude)], popup=popup_text).add_to(m)

        m.save(output_html)
        print(f"Map saved as {output_html}")
    else:
        print("No valid data to visualize.")

# Main Workflow
def main(image_file):
    # Step 1: Extract GPS coordinates
    lat, lon = extract_gps(image_file)
    if lat is None or lon is None:
        print("Skipping image due to missing GPS data.")
        return

    print(f"GPS Coordinates: Latitude={lat}, Longitude={lon}")

    # Step 2: Detect potholes
    detections, annotated_image_path = detect_potholes(image_file)
    if not detections:
        print("No potholes detected.")
        return

    # Step 3: Log to CSV
    for _, _, _, _, confidence in detections:
        log_to_csv(lat, lon, confidence, annotated_image_path)

    # Step 4: Create Map
    create_map(csv_file, html_map_file)

if __name__ == "__main__":
    # Example image path
    image_file = "/home/shashank/Desktop/Mini_project /IMG_9394.jpg"
    main(image_file)


GPS Coordinates: Latitude=12.940794444444444, Longitude=77.56542222222222

image 1/1 /home/shashank/Desktop/Mini_project /IMG_9394.jpg: 480x640 1 pothole, 107.5ms
Speed: 4.1ms preprocess, 107.5ms inference, 2.1ms postprocess per image at shape (1, 3, 480, 640)
Logging to CSV: [12.940794444444444, 77.56542222222222, 0.5490937829017639, '2025-02-15 06:39:45', 'annotated_images/annotated_20250215_063945.jpg']
Row: ['12.940794444444444', '77.56542222222222', '0.5490937829017639', '2025-02-15 06:39:45', 'annotated_images/annotated_20250215_063945.jpg']
Map saved as pothole_map2.html
